In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score, recall_score, roc_auc_score, precision_recall_curve
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the dataset

In [ ]:
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
X = df.iloc[:,:-1].values
Y = df.iloc[:, -1].values
df.head(3)

# Data information

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().any()

In [ ]:
df['Outcome'].value_counts()

# Accurancy, Precision and Recall

In [ ]:
def get_dataset_eval(y_test, pred=None, pred_prob=None):
    eval_dic = {}
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    roc_auc = roc_auc_score(y_test, pred_prob)
    
    print("Confusion Matrix")
    print(confusion)
    
    print("Accuracy: {0:.4f}, Precision: {1:.4f}, Recall: {2:.4f}, F1: {3:.4f}, AUC: {4:.4f}".format(accuracy, precision, recall, f1, roc_auc))
    eval_dic['Accuracy'] = accuracy
    eval_dic['Precision'] = precision
    eval_dic['Recall'] = recall
    eval_dic['F1'] = f1
    eval_dic['ROC AUC'] = roc_auc
    
    return eval_dic


# Visualization for Precision and Recall

In [ ]:
def precision_recall_curve_plot(y_test=None, pred_prob=None):
    
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_prob)
    
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[:threshold_boundary], linestyle='--', label='Precision', color='red')
    plt.plot(thresholds, recalls[:threshold_boundary], label='Recall', color='blue')
    
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))
    plt.xlabel("threshold")
    plt.ylabel("Precision and Recall")
    plt.legend()
    plt.show()

## Splitting the dataset into the Training set and Test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## Training the Multiple Linear Regression model on the Training set

In [ ]:
from sklearn.linear_model import LogisticRegression
regressor = LogisticRegression()
regressor.fit(X_train, y_train)

# Predicting the Test set results

In [ ]:
y_pred = regressor.predict(X_test)
pred_proba = regressor.predict_proba(X_test)[:,1]

In [ ]:
# Accuracy, Precision, Recall
get_dataset_eval(y_test, y_pred, pred_proba)

In [ ]:
# precision recall curve
precision_recall_curve_plot(y_test, pred_proba)